In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
response = requests.get('https://www.edaily.co.kr/search/news/?source=total&keyword=%ed%99%98%ec%9c%a8&include=&exclude=&jname=&start=20240901&end=20240903&sort=latest&date=pick&exact=false&page=1')

soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
title = soup.select_one('#newsList li').text
title

"[미리보는 이데일리 신문]'물류센터 공급절벽 곧 온다' 선점 나선 외국 투자자들"

In [10]:
contents = soup.select('ul.newsbox_texts li')
contents

[<li>[미리보는 이데일리 신문]'물류센터 공급절벽 곧 온다' 선점 나선 외국 투자자들</li>,
 <li>[이데일리 김가영 기자] 다음은 4일자 이데일리 신문 주요 뉴스다.△1면-‘물류센터 공급절벽 곧 온다’ 선점 나선 외국 투자자들-국정원 보안규제에 발목잡힌 토종 클라우드-은행 임직원 횡령 7년간 1536억…사고 절반은 우리은행-8월 물가 2%↑…3년 5개월 만에 최저△종합-[사설]소비자물가 안정권…피벗 선언할 때다-[사설]뜬금없는 계엄 공방, 협치 복원 말뿐인가-[르포]“韓 FA-50 배우자” 폴란드 공군 열공 KAI 인력 상주, 정비·훈련 밀착지원△음주운전 공화국-엄벌 가로막는 ‘교특법’에…음주운전 사망사고 4건 중 3건은 집행유예-“관대한 처벌 근거 교특법, 없애거나 개정해야”-[기고]“음주운전 경각심 심어줘야…교특법 최고형량 10년으로 상향을”△금리인하기 안전자산 투자 전략-중동 위기·약달러 전환…주춤한 금값, 길게 보면 오른다-환율 변동 최소화…‘환헤지 ETF’ 뜬다-안정적 배당수익률 보장…‘리츠·채권’ 자금 유입 지속△몸값 들썩이는 물류센터-매년 264만㎡ 임차수요 발생하는데…착공 없어 2026년부터 ‘공급부족’-경매 나온 물류센터 사들이는 운용사들-실탄 풍부한 외국계 투자자, 알짜 물류센터 눈독△종합-‘딥페이크 범죄자’ 신상공개 요구 빗발…미검증 ‘가해자 정보’ 횡행 부작용-“좋은 일자리 창출은 기업 의무” 67년째 이어진 삼성 정기공채-소비자물가, 정부 목표 도달…10월 금리 인하 ‘청신호’-횡령액 ‘최고’ 환수율 ‘최저’…우리은행 겨눈 금감원△정치-계엄령 띄우기 열 올리는 민주당…명확한 근거 못 대며 군불만-“李 잦은 재판은 법정연금”…사법리스크 엄호 나선 친명-“北 주민 충성심은 옛말, 간부들도 체제에 환멸감”-구미 산단 간 한동훈 “반도체 산업 심장으로”-“트럼프 당선되면 美 안보우산 약화 가능성”△경제-한은의 경고 “집값 버블 심각…정책효과 없다면 금리 고려”-공공기관 4곳 중 1곳 직원에 더 싸게 대출-필리핀 가사관리사, 미지급 수

In [530]:
date_range = pd.date_range('2023-01-01', '2024-8-31', freq='D')
keyword = '환율'
# 하루에 하나씩 날짜 출력
dates = []

single_date = []
urls = []
for date in date_range:
    single_date = date.strftime('%Y%m%d')
    urls.append(f'https://www.edaily.co.kr/search/news/?source=total&keyword={keyword}&include=&exclude=&jname=&start={single_date}&end={single_date}&sort=latest&date=pick&exact=false&page=')


f_urls = []
for url in urls:
    page = 1
    while True:
        current_url = url + str(page)
        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cont = soup.select('ul.newsbox_texts li')
        if len(cont) == 0:
            break
        f_urls.append(current_url)
        page += 1

contents = []
reg_date = []
for url in f_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cont = soup.select('ul.newsbox_texts li')
    date = soup.select('div.author_category')
    contents.extend(cont)
    reg_date.extend(date)

    
        
titles = []
articles = []

num = 0
for ct in contents:
    if num % 2 == 0:
        titles.extend(ct)
            
    else:
        articles.extend(ct)
    num += 1

tmp_articles = []
for art in articles:
    tmp_articles.append(art.text.split('기자]',1)[-1])


n = 0
f_reg_date = []
for dt in reg_date:
    f_reg_date.append(dt.text.split('\r\n')[1].strip())

# open_url = soup.select('#newsList a')
# f_open_url = []
# for ou in open_url:
#     f_open_url.append(ou.attrs['href'])


# 만약 길이가 다르면, 길이를 맞추기 위해 추가적인 작업이 필요함
# 예를 들어, 짧은 리스트에 None을 추가

print('length_titles:',len(titles))
print('reg_date',len(f_reg_date))
print('articles',len(tmp_articles))


min_length = min(len(titles), len(f_reg_date), len(tmp_articles))

titles = titles[:min_length]
f_reg_date = f_reg_date[:min_length]
tmp_articles = tmp_articles[:min_length]

# # 그 다음 데이터 프레임을 생성
data = {'titles': titles, 'media':'이데일리', 'reg_date': f_reg_date, 'article' : tmp_articles }

df = pd.DataFrame(data)

    


length_titles: 6196
reg_date 6196
articles 6196


In [533]:
list = [df2005,df2006, df2007,df2008, df2009_10, df2011_12, df2013_14, df2015_16, df2017_18, df2019_20, df2021_22, df2023_24]

In [538]:
edaily_df = pd.concat(list, axis = 0, ignore_index = True)

edaily_df.to_json('edaily_2005_24.json')

edaily_df.to_csv('edaily_2005_24.csv')

In [543]:
edaily_df.to_csv('/Users/sungminhong/Documents/LLM_Service/BOK/edaily_2005_24.csv',index=True)

In [386]:
titles

['“달러 말고 위안 주세요”…달러대비 위안화 가치 15개월만 최고',
 "농심, 해외사업 강화해 '라면명가' 지위 굳힌다",
 '‘빅컷’ 결정할 美고용 지표…환율 변동성 확대 경계[주간외환전망]',
 '日증시, 상승마감…장중엔 1달만에 3만 9000선 회복',
 'NH투자증권 “한미 금리차 개선, 원화 호재 아냐…하반기 환율 상승”',
 '강달러 완화·외국인 증시 유입…환율, 1338.4원 상승 그쳐[외환마감]',
 '[이지혜의 뷰]삼양식품 불닭볶음면 여전히 맵다',
 '강달러 속 위안화 반등…환율, 장중 1340원대로 상승[외환분석]',
 '대신증권, MTS ‘대신 사이보스’·‘크레온’ 개편',
 '예상 부합한 PCE 물가…환율, 1337원으로 상승 출발',
 'NDF, 1334.4원/1334.8원…1.3원 상승',
 '물가 순항 확인한 PCE…환율 1330원 후반대 상승[외환브리핑]',
 '“9월 증시, 계절적 약세…현금 비중 확대”',
 '"8월 물가상승률 2.0%"…한은 목표 수준 도달[물가폴]',
 '"2차전지 괜히 팔았네"…개미만 8월에 손실 봤다',
 "[미리보는 이데일리 신문]'물류센터 공급절벽 곧 온다' 선점 나선 외국 투자자들",
 '아연價 더 오르나…고려아연, 하반기 실적 기대감↑',
 '\'비수기\' 7월에도 항공화물 날았다…"하반기도 고공행진"',
 '中企 운전자금 금리 4%대로 낮아져…22개월만',
 '美고용 경계 속 외인 증시 매도…환율 1340원대 안착[외환마감]',
 '한산한 장 속 위안화 약세 연동…환율, 장중 1343원으로 상승[외환분석]',
 '한총리 "물가 드디어 2%로 안정…금리 내릴 수 있는 여지 생겨"',
 'KB證, 투자전략 담은 ‘다이렉트인덱싱 몰’ 오픈',
 '美노동절 휴장 영향…환율, 1338원 약보합 출발',
 'NDF, 1335.5원/1335.9원…보합',
 '신한운용, ‘미국장기국채분할매수 목표전환형 펀드’ 모집',
 '미국 휴장 속 이벤트 부재…환율 1330원 레인지 지속[외환브리핑]',
 '한국가스공

In [352]:
f_reg_date

['2024.09.01',
 '2024.09.01',
 '2024.09.01',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.02',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03',
 '2024.09.03']

In [347]:
len(tmp_articles)

30

In [353]:
titles

['“달러 말고 위안 주세요”…달러대비 위안화 가치 15개월만 최고',
 "농심, 해외사업 강화해 '라면명가' 지위 굳힌다",
 '‘빅컷’ 결정할 美고용 지표…환율 변동성 확대 경계[주간외환전망]',
 '日증시, 상승마감…장중엔 1달만에 3만 9000선 회복',
 'NH투자증권 “한미 금리차 개선, 원화 호재 아냐…하반기 환율 상승”',
 '강달러 완화·외국인 증시 유입…환율, 1338.4원 상승 그쳐[외환마감]',
 '[이지혜의 뷰]삼양식품 불닭볶음면 여전히 맵다',
 '강달러 속 위안화 반등…환율, 장중 1340원대로 상승[외환분석]',
 '대신증권, MTS ‘대신 사이보스’·‘크레온’ 개편',
 '예상 부합한 PCE 물가…환율, 1337원으로 상승 출발',
 'NDF, 1334.4원/1334.8원…1.3원 상승',
 '물가 순항 확인한 PCE…환율 1330원 후반대 상승[외환브리핑]',
 '“9월 증시, 계절적 약세…현금 비중 확대”',
 '"8월 물가상승률 2.0%"…한은 목표 수준 도달[물가폴]',
 '"2차전지 괜히 팔았네"…개미만 8월에 손실 봤다',
 "[미리보는 이데일리 신문]'물류센터 공급절벽 곧 온다' 선점 나선 외국 투자자들",
 '아연價 더 오르나…고려아연, 하반기 실적 기대감↑',
 '\'비수기\' 7월에도 항공화물 날았다…"하반기도 고공행진"',
 '中企 운전자금 금리 4%대로 낮아져…22개월만',
 '美고용 경계 속 외인 증시 매도…환율 1340원대 안착[외환마감]',
 '한산한 장 속 위안화 약세 연동…환율, 장중 1343원으로 상승[외환분석]',
 '한총리 "물가 드디어 2%로 안정…금리 내릴 수 있는 여지 생겨"',
 'KB證, 투자전략 담은 ‘다이렉트인덱싱 몰’ 오픈',
 '美노동절 휴장 영향…환율, 1338원 약보합 출발',
 'NDF, 1335.5원/1335.9원…보합',
 '신한운용, ‘미국장기국채분할매수 목표전환형 펀드’ 모집',
 '미국 휴장 속 이벤트 부재…환율 1330원 레인지 지속[외환브리핑]',
 '한국가스공

In [391]:
response = requests.get('https://www.edaily.co.kr/search/news/?source=total&keyword=%ea%b2%bd%ec%a0%9c&include=&exclude=&jname=&start=20240901&end=20240901&sort=latest&date=pick&exact=false&page=2')
soup = BeautifulSoup(response.text, 'html.parser')

In [394]:
len(soup.select('div.newsbox_04 a'))

38

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('/Users/sungminhong/Documents/LLM_Service/BOK/edaily_2005_24.json', encoding='utf-8')

In [3]:
df['article'] = [{'keyword': art} for art in df['article']]

In [4]:
df['id'] = None

In [5]:
df['reg_date'] = df['reg_date'].rename('date')

In [6]:
df.rename(columns={'titles': 'title', 'reg_date' : 'date', 'article' :'sentences'}, inplace=True)

In [7]:
from datetime import datetime

In [8]:
df['reg_date'] = datetime.now()

In [9]:
df['document_type'] = 'news_article'

In [10]:
df['sentiment_score_market'] = None

In [11]:
df

,title,media,date,sentences,id,reg_date,document_type,sentiment_score_market
0,1월증시 `조정시 매수우위 전략` 바람직-LG,이데일리,2005.01.01,{'keyword': '1일 LG증권은 1월 증시가 전반적인 상승 기조는 유지될 것...,None,2024-09-09 02:22:26.405373,news_article,None
1,작년 수출 2500억불 돌파..사상최고(종합),이데일리,2005.01.01,"{'keyword': ' 지난해 수출이 2500억달러를 돌파, 사상최고치로 세계 1...",None,2024-09-09 02:22:26.405373,news_article,None
2,역외환율 호가 1035원선 하락..거래 전무,이데일리,2005.01.01,{'keyword': ' 현물환에 이어 역외환율도 1030원선으로 떨어졌다. ...,None,2024-09-09 02:22:26.405373,news_article,None
3,(월가시각)내일에는 내일의 태양이,이데일리,2005.01.01,{'keyword': '[뉴욕=edaily 안근모특파원] 2004년 마지막 거래일의...,None,2024-09-09 02:22:26.405373,news_article,None
4,(가판분석)1월3일자 조간신문 주요기사,이데일리,2005.01.02,{'keyword': ' ◇ 헤드라인 -경향: 독립운동史 `이념 굴레`...,None,2024-09-09 02:22:26.405373,news_article,None
...,...,...,...,...,...,...,...,...
119398,"NDF, 1331.3원/1331.7원…0.95원 상승",이데일리,2024.08.30,{'keyword': ' 간밤 뉴욕 차액결제선물환(NDF) 시장에서 거래된 원·달러...,None,2024-09-09 02:22:26.405373,news_article,None
119399,달러 강세 속 월말 수급…환율 1330원대 공방[외환브리핑],이데일리,2024.08.30,{'keyword': ' 원·달러 환율은 1330원대에서 공방이 예상된다. 미국 경...,None,2024-09-09 02:22:26.405373,news_article,None
119400,"삼양식품, 주가 조정은 저가매수 기회-키움",이데일리,2024.08.30,{'keyword': ' 키움증권은 30일 삼양식품(003230)에 대해 최근의 주...,None,2024-09-09 02:22:26.405373,news_article,None
119401,"더블유게임즈, 하반기 신성장 동력 확보 전망…저평가 요인 제거 기대-키움",이데일리,2024.08.30,{'keyword': ' 키움증권은 30일 더블유게임즈(192080)에 대해 하반기...,None,2024-09-09 02:22:26.405373,news_article,None


In [12]:
df = df[['id','document_type','date','title','media','sentences', 'sentiment_score_market', 'reg_date']]

In [13]:
df

,id,document_type,date,title,media,sentences,sentiment_score_market,reg_date
0,None,news_article,2005.01.01,1월증시 `조정시 매수우위 전략` 바람직-LG,이데일리,{'keyword': '1일 LG증권은 1월 증시가 전반적인 상승 기조는 유지될 것...,None,2024-09-09 02:22:26.405373
1,None,news_article,2005.01.01,작년 수출 2500억불 돌파..사상최고(종합),이데일리,"{'keyword': ' 지난해 수출이 2500억달러를 돌파, 사상최고치로 세계 1...",None,2024-09-09 02:22:26.405373
2,None,news_article,2005.01.01,역외환율 호가 1035원선 하락..거래 전무,이데일리,{'keyword': ' 현물환에 이어 역외환율도 1030원선으로 떨어졌다. ...,None,2024-09-09 02:22:26.405373
3,None,news_article,2005.01.01,(월가시각)내일에는 내일의 태양이,이데일리,{'keyword': '[뉴욕=edaily 안근모특파원] 2004년 마지막 거래일의...,None,2024-09-09 02:22:26.405373
4,None,news_article,2005.01.02,(가판분석)1월3일자 조간신문 주요기사,이데일리,{'keyword': ' ◇ 헤드라인 -경향: 독립운동史 `이념 굴레`...,None,2024-09-09 02:22:26.405373
...,...,...,...,...,...,...,...,...
119398,None,news_article,2024.08.30,"NDF, 1331.3원/1331.7원…0.95원 상승",이데일리,{'keyword': ' 간밤 뉴욕 차액결제선물환(NDF) 시장에서 거래된 원·달러...,None,2024-09-09 02:22:26.405373
119399,None,news_article,2024.08.30,달러 강세 속 월말 수급…환율 1330원대 공방[외환브리핑],이데일리,{'keyword': ' 원·달러 환율은 1330원대에서 공방이 예상된다. 미국 경...,None,2024-09-09 02:22:26.405373
119400,None,news_article,2024.08.30,"삼양식품, 주가 조정은 저가매수 기회-키움",이데일리,{'keyword': ' 키움증권은 30일 삼양식품(003230)에 대해 최근의 주...,None,2024-09-09 02:22:26.405373
119401,None,news_article,2024.08.30,"더블유게임즈, 하반기 신성장 동력 확보 전망…저평가 요인 제거 기대-키움",이데일리,{'keyword': ' 키움증권은 30일 더블유게임즈(192080)에 대해 하반기...,None,2024-09-09 02:22:26.405373


In [14]:
df.to_json('edaily_0524.json', orient='records', lines=True, force_ascii=False)

In [15]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from sqlalchemy import create_engine



In [ ]:
df.to_sql('edaily', con=engine, if_exists='append', index=False)

In [18]:
df_tmp = df.head(1)

In [19]:
df_tmp

,id,document_type,date,title,media,sentences,sentiment_score_market,reg_date
0,None,news_article,2005.01.01,1월증시 `조정시 매수우위 전략` 바람직-LG,이데일리,{'keyword': '1일 LG증권은 1월 증시가 전반적인 상승 기조는 유지될 것...,None,2024-09-09 02:22:26.405373


In [ ]:
user = 'sungmin'  # 사용자 이름
password = '0000'  # 비밀번호
host = 'paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com'  # 호스트
database = 'PAPER_PROJECT'  # 데이터베이스 이름

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# 데이터프레임을 데이터베이스에 저장
df_tmp.to_sql('text_analysis', con=engine, if_exists='append', index=False)

print("데이터프레임이 MySQL 데이터베이스에 저장되었습니다.")

In [24]:
df.dtypes

id                                object
document_type                     object
date                              object
title                             object
media                             object
sentences                         object
sentiment_score_market            object
reg_date                  datetime64[us]
dtype: object

In [25]:
df['document_type'] = 'news_edaily'

In [30]:
df['date'] = pd.to_datetime(df['date'])

In [31]:
df['sentiment_score_market'] = pd.to_numeric(df['sentiment_score_market'])

In [36]:
df_tmp = df.head(1)

In [38]:
df_tmp

,id,document_type,date,title,media,sentences,sentiment_score_market,reg_date
0,NaN,news_edaily,2005-01-01,1월증시 `조정시 매수우위 전략` 바람직-LG,이데일리,{'keyword': '1일 LG증권은 1월 증시가 전반적인 상승 기조는 유지될 것...,NaN,2024-09-09 02:22:26.405373


In [41]:
df['id'] = 0

In [43]:
df['sentiment_score_market'] = 0

In [44]:
df

,id,document_type,date,title,media,sentences,sentiment_score_market,reg_date,4971227152
0,0,news_edaily,2005-01-01,1월증시 `조정시 매수우위 전략` 바람직-LG,이데일리,{'keyword': '1일 LG증권은 1월 증시가 전반적인 상승 기조는 유지될 것...,0,2024-09-09 02:22:26.405373,0
1,0,news_edaily,2005-01-01,작년 수출 2500억불 돌파..사상최고(종합),이데일리,"{'keyword': ' 지난해 수출이 2500억달러를 돌파, 사상최고치로 세계 1...",0,2024-09-09 02:22:26.405373,0
2,0,news_edaily,2005-01-01,역외환율 호가 1035원선 하락..거래 전무,이데일리,{'keyword': ' 현물환에 이어 역외환율도 1030원선으로 떨어졌다. ...,0,2024-09-09 02:22:26.405373,0
3,0,news_edaily,2005-01-01,(월가시각)내일에는 내일의 태양이,이데일리,{'keyword': '[뉴욕=edaily 안근모특파원] 2004년 마지막 거래일의...,0,2024-09-09 02:22:26.405373,0
4,0,news_edaily,2005-01-02,(가판분석)1월3일자 조간신문 주요기사,이데일리,{'keyword': ' ◇ 헤드라인 -경향: 독립운동史 `이념 굴레`...,0,2024-09-09 02:22:26.405373,0
...,...,...,...,...,...,...,...,...,...
119398,0,news_edaily,2024-08-30,"NDF, 1331.3원/1331.7원…0.95원 상승",이데일리,{'keyword': ' 간밤 뉴욕 차액결제선물환(NDF) 시장에서 거래된 원·달러...,0,2024-09-09 02:22:26.405373,0
119399,0,news_edaily,2024-08-30,달러 강세 속 월말 수급…환율 1330원대 공방[외환브리핑],이데일리,{'keyword': ' 원·달러 환율은 1330원대에서 공방이 예상된다. 미국 경...,0,2024-09-09 02:22:26.405373,0
119400,0,news_edaily,2024-08-30,"삼양식품, 주가 조정은 저가매수 기회-키움",이데일리,{'keyword': ' 키움증권은 30일 삼양식품(003230)에 대해 최근의 주...,0,2024-09-09 02:22:26.405373,0
119401,0,news_edaily,2024-08-30,"더블유게임즈, 하반기 신성장 동력 확보 전망…저평가 요인 제거 기대-키움",이데일리,{'keyword': ' 키움증권은 30일 더블유게임즈(192080)에 대해 하반기...,0,2024-09-09 02:22:26.405373,0


In [54]:
df_tmp =df.head(1)

In [70]:
user = 'sungmin'  # 사용자 이름
password = '0000'  # 비밀번호
host = 'paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com'  # 호스트
database = 'PAPER_PROJECT'  # 데이터베이스 이름

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

# 데이터프레임을 데이터베이스에 저장
df.to_sql('text_analysis', con=engine, if_exists='append', index=False)

print("데이터프레임이 MySQL 데이터베이스에 저장되었습니다.")

데이터프레임이 MySQL 데이터베이스에 저장되었습니다.
